In [16]:
username=12

In [1]:
from river import optim
from river import reco
import pandas as pd
from json import loads, dumps

C:\Users\sashank\AppData\Local\Temp\ipykernel_50908\1539750421.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
dataset_path = r"C:\Users\sashank\Downloads\output_with_ratings (3).csv"
df_org = pd.read_csv(dataset_path)
df = df_org.drop(['album_name','track_name','artist_name'],axis =1)
df

,pid,unique_id,rating
0,0,1,3
1,0,2,4
2,0,3,1
3,0,4,2
4,0,5,1
...,...,...,...
67498,999,2040,1
67499,999,30047,4
67500,999,30048,3
67501,999,352,3


In [4]:
df = df[0:5000]

In [5]:
#input
songID_list = df['unique_id'].tolist()

In [8]:
df_dict = df.to_dict(orient='records')


In [9]:
dataset = tuple(({'user': row['pid'], 'item': row['unique_id']}, row.pop('rating')) for row in df_dict)
dataset

(({'user': 0, 'item': 1}, 3),
 ({'user': 0, 'item': 2}, 4),
 ({'user': 0, 'item': 3}, 1),
 ({'user': 0, 'item': 4}, 2),
 ({'user': 0, 'item': 5}, 1),
 ({'user': 0, 'item': 6}, 3),
 ({'user': 0, 'item': 7}, 4),
 ({'user': 0, 'item': 8}, 4),
 ({'user': 0, 'item': 9}, 2),
 ({'user': 0, 'item': 10}, 4),
 ({'user': 0, 'item': 11}, 3),
 ({'user': 0, 'item': 12}, 4),
 ({'user': 0, 'item': 13}, 4),
 ({'user': 0, 'item': 14}, 4),
 ({'user': 0, 'item': 15}, 3),
 ({'user': 0, 'item': 16}, 2),
 ({'user': 0, 'item': 17}, 4),
 ({'user': 0, 'item': 18}, 2),
 ({'user': 0, 'item': 19}, 2),
 ({'user': 0, 'item': 20}, 1),
 ({'user': 0, 'item': 21}, 1),
 ({'user': 0, 'item': 22}, 3),
 ({'user': 0, 'item': 23}, 2),
 ({'user': 0, 'item': 24}, 1),
 ({'user': 0, 'item': 25}, 3),
 ({'user': 0, 'item': 26}, 3),
 ({'user': 0, 'item': 27}, 2),
 ({'user': 0, 'item': 28}, 4),
 ({'user': 0, 'item': 29}, 1),
 ({'user': 0, 'item': 30}, 1),
 ({'user': 0, 'item': 31}, 4),
 ({'user': 0, 'item': 32}, 4),
 ({'user': 0, 'it

In [10]:
model = reco.BiasedMF(
    n_factors=10,
    bias_optimizer=optim.SGD(0.025),
    latent_optimizer=optim.SGD(0.025),
    latent_initializer=optim.initializers.Normal(mu=0., sigma=0.1, seed=71)
)

In [11]:
for x, y in dataset:
    model.learn_one(**x, y=y)

In [13]:
a = model.rank(user = username, items = songID_list)

In [17]:
print(type(a))

<class 'list'>


In [25]:
a = a[0:10]
recommended_list=[]

In [26]:
for i in a:
  if i==0:
    recommended_list=(df_org[df_org['unique_id']==i])
  else:
    recommended_list.append(df_org[df_org['unique_id']==i])
    

In [37]:
songs = []
artists = []
for i in a:
  rows = df_org[df_org['unique_id']==i]
  b = rows['track_name'].unique()
  c = rows['artist_name'].unique()
  artists.append(c[0])
  songs.append(b[0])
print(songs)
artists


['Latch', 'Cruise', 'Blunt Blowin', 'Jesus Saves', "'Till I Collapse", 'Somebody That I Used To Know', 'Push', 'White Walls (feat. ScHoolboy Q, Hollis)', 'Capsize', 'Knives']


['Disclosure',
 'Florida Georgia Line',
 'Lil Wayne',
 'Daryl Coley',
 'Eminem',
 'Gotye',
 'Fog Lake',
 'Macklemore & Ryan Lewis',
 'FRENSHIP',
 'Matoma']

In [43]:
input_list=[{"track_name":song,"artist_name":artist}for song,artist in zip(songs,artists)]
input_list


[{'track_name': 'Latch', 'artist_name': 'Disclosure'},
 {'track_name': 'Cruise', 'artist_name': 'Florida Georgia Line'},
 {'track_name': 'Blunt Blowin', 'artist_name': 'Lil Wayne'},
 {'track_name': 'Jesus Saves', 'artist_name': 'Daryl Coley'},
 {'track_name': "'Till I Collapse", 'artist_name': 'Eminem'},
 {'track_name': 'Somebody That I Used To Know', 'artist_name': 'Gotye'},
 {'track_name': 'Push', 'artist_name': 'Fog Lake'},
 {'track_name': 'White Walls (feat. ScHoolboy Q, Hollis)',
  'artist_name': 'Macklemore & Ryan Lewis'},
 {'track_name': 'Capsize', 'artist_name': 'FRENSHIP'},
 {'track_name': 'Knives', 'artist_name': 'Matoma'}]

In [29]:
print(recommended_list[0])

       pid  unique_id  rating album_name track_name artist_name
438      7        428       3     Settle      Latch  Disclosure
4933    86        428       5     Settle      Latch  Disclosure
4935    86        428       4     Settle      Latch  Disclosure
4940    86        428       4     Settle      Latch  Disclosure
4941    86        428       5     Settle      Latch  Disclosure
4942    86        428       4     Settle      Latch  Disclosure
4943    86        428       5     Settle      Latch  Disclosure
5555    95        428       4     Settle      Latch  Disclosure
6338   103        428       1     Settle      Latch  Disclosure
6476   105        428       3     Settle      Latch  Disclosure
13798  214        428       1     Settle      Latch  Disclosure
16219  250        428       4     Settle      Latch  Disclosure
25230  396        428       4     Settle      Latch  Disclosure
31252  468        428       2     Settle      Latch  Disclosure
35353  529        428       5     Settle